In [ ]:
# import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()

import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from ecig_parsing import *


In [ ]:
def get_html(url, clicked=False, closed=False, elements=True):
    # print('GET', url)
    driver = None
    try:
#         # Set up headless mode for Firefox
        options = Options()
        options.add_argument('--headless')  # Enable headless mode properly

        # Start the WebDriver in headless mode
        driver = webdriver.Firefox(options=options)
#         driver = webdriver.Firefox()


        # Start the WebDriver in headless mode
        driver.get(url)
        last_n = 0
        hrefs = set()
        button_id = 'ac-ag-yes-button'

        html = driver.page_source
        time.sleep(1)
        same_count = 0
        if not clicked:
            wait = WebDriverWait(driver, 10)  # 10 seconds timeout
            button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))  # Use ID, XPATH, or other locator

            # Click the button
            button.click()
            clicked = True
#         if not closed:
#             wait = WebDriverWait(driver, 10)  # 10 seconds timeout
#             button = wait.until(EC.element_to_be_clickable((By.ID, 'omnisend-form-65006a9819ed2c118d9a9352-close-action')))  # Use ID, XPATH, or other locator

#             # Click the button
#             button.click()
#             closed = True
    

        # Set a delay for dynamic content loading
        SCROLL_PAUSE_TIME = 0.5  # Shorter pause for smoother scroll

        # Scroll incrementally by a small step (e.g., 500 pixels)
        scroll_increment = 500

        # Get the initial scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        first_height = last_height
        # print(last_height)

        while True:
            # Scroll down by the increment
            driver.execute_script(f"window.scrollBy(0, {scroll_increment});")

            # Wait for the page to load new content
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate the new scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # Check if new elements have appeared on the screen
            # You can check specific elements like this:
            # elements = driver.find_elements(By.CLASS_NAME, "your-element-class")

            # If the scroll height has not changed, break the loop
            #print(same_count, new_height, last_height)
            if new_height == last_height :
                same_count += 1
                if same_count >= 3:
                    break
            else:
                same_count = 0

            # Update the last height
            last_height = new_height
            
            html = driver.page_source
            #print(html)
        

    except Exception as ex:
        print(ex)
        html = ''
    finally:
        if driver:
            driver.close()




    return html


In [ ]:
import json
import os

# Specify the file path
json_file_path = 'vapewh.json'

# Check if the file exists
if os.path.exists(json_file_path):
    # Open and read the JSON file
    with open(json_file_path, 'r') as file:
        try:
            found = json.load(file)
            print("JSON data:", len(found.keys()), 'items')
        except json.JSONDecodeError:
            print("Error: The file contains invalid JSON.")
else:
    found = dict()
    print(f"File '{json_file_path}' does not exist.")

In [ ]:
# https://vapewh.com/collections/all-disposable-e-cigs?page=2

In [ ]:
links = [
    "/collections/vape-deals",
    "/collections/all-disposable-e-cigs",
    "/collections/3-nicotine-disposable-vapes",
    "/collections/newest-arrivals-1",
    "/collections/smart-vapes",
    "/collections/all-lower-nicotine-vapes",
    "/collections/vapes-with-battery-indicators",
    "/products/geek-bar-pulse",
    "/collections/raz",
    "/collections/elf-bar-disposable-vapes",
    "/collections/realio",
    "/products/air-bar-diamond-box-vape-1",
    "/collections/bc5000",
    "/products/geek-bar-pulse-x",
    "/collections/lost-mary",
    "/collections/ijoy",
    "/collections/again-u-bar",
    "/collections/kado-bars",
    "/products/fifty-bar-disposable-6500-puffs",
    "/products/priv-bar-turbo",
    "/products/nexa-n20000",
    "/collections/fire",
    "/products/morfbar-20k-puff-disposable",
    "/collections/monster-bars",
    "/collections/hqd",
    "/collections/blankz-disposables",
    "/collections/spaceman",
    "/collections/tobacco-flavors",
    "/collections/mint-spearmint",
    "/collections/fruity-flavor",
    "/collections/strawberry-flavor-vapes",
    "/collections/banana-flavor-vapes",
    "/collections/dessert-flavors",
    "/collections/drink-flavors",
    "/collections/candy-flavors",
    "/collections/iced",
    "/collections/non-iced",
    "/collections/liquid",
    "/collections/sub-ohm-e-liquids",
    "/collections/nicotine-salts",
    "/collections/pod-devices",
    "/collections/pod-devices-1",
    "/collections/tanks-coils",
    "/collections/replacement-pods",
    "/collections/ri-products"
]


In [6]:
from collections import Counter
import csv
product_list = list()
all_headers = list()
header_counter = Counter(all_headers)
product_tags = set()


def save():
    print('SAVING...')
    with open('vapewh_scrape.csv', mode='w') as file:
        # Create a DictWriter object
        writer = csv.DictWriter(file, fieldnames=product_list[0].keys())

        # Write the header (column names)
        writer.writeheader()

        # Write the rows (each dictionary in the data list)
        writer.writerows(product_list)
        
    with open(json_file_path, 'w') as file:
        json.dump(found, file, indent=4) 


max_pages = 25
found_max = False
for link_index,l in enumerate(links):
    page = 1
    while True:
        if page > max_pages:
            break
        if l.startswith('https'):
            main_url = f'{l}?page={page}'
        else:
            main_url = f'https://vapewh.com{l}?page={page}'
            
        try:
            if main_url in found:
                req_text = found[main_url]
            else:
                req_text = get_html(main_url)
                found[main_url] = req_text

            soup = BeautifulSoup(req_text)
            prods = soup.find_all('div', {'class': 'box product'})
            

        except Exception as ex:
            print(ex, 'FAILED', main_url)
            break
           
        if not found_max:
            try:
                pagination = soup.find('ul', class_='pagination')

                # Get the last page number by finding the last <li> with an <a> tag that is not empty
                last_page = None
                for li in pagination.find_all('li'):
                    a_tag = li.find('a')
                    if a_tag and a_tag.text.strip().isdigit():
                        max_pages = int(a_tag.text.strip())
                        found_max = True
            except Exception as ex:
                max_pages = 25
            
        total_items = len(prods)
        print('LINK INDEX', link_index, 'OF', len(links), ' -- PAGE', page, 'OF', max_pages, f'({total_items})', main_url)

            
        page += 1
        

        
        if total_items == 0:
            break
            
        
        for psoup in prods:
            txt = psoup.get_text().strip()
            if txt == '':
                total_items -= 1
                continue
            sku = psoup['data-sku'] 
            product_name = psoup.find('a', class_='title').text.strip() if soup.find('a', class_='title') else None
            product_url = psoup.find('a', class_='title')['href'].strip() if soup.find('a', class_='title') else None
            
            if not product_url:
                continue
            image_url = psoup.find('img', class_='product_card__image')['src'] if soup.find('img', class_='product_card__image') else None
            alt_text = psoup.find('img', class_='product_card__image')['alt'] if soup.find('img', class_='product_card__image') else None
            money  = psoup.find('span', class_='money')
            if money:
                price = money.text 
            else:
                price = None
            rating = psoup.find('div', class_='loox-rating')['data-rating'] if soup.find('div', class_='loox-rating') else None
            raters = psoup.find('div', class_='loox-rating')['data-raters'] if soup.find('div', class_='loox-rating') else None
            variation_elem = psoup.find('span', class_='vendor')
            if variation_elem:
                variation = variation_elem.text 

#             # Display the extracted elements
#             print(f"SKU: {sku}")
#             print(f"Product Name: {product_name}")
#             print(f"Product URL: {product_url}")
#             print(f"Image URL: {image_url}")
#             print(f"Alt Text: {alt_text}")
#             print(f"Price: {price}")
#             print(f"Rating: {rating}")
#             print(f"Number of Raters: {raters}")
#             print(f"Variation: {variation}")
            
            if product_url.startswith('http'):
                full_link = product_url.strip()
            else:
                full_link = f'https://vapewh.com{product_url}'.strip()
                
            if image_url.startswith('//'):
                image_url = 'http:' + image_url
                
            tag = full_link.split('/')[-1]
            if tag in product_tags:
                continue
                
            try:
            
                if full_link in found:
                    reqtxt = found[full_link]
                else:
                    # print(full_link)
                    reqtxt = get_html(full_link, elements=False)
                    found[full_link] = reqtxt
            except Exception as ex:
                print(ex, 'FAILED', main_url)
                continue
                


                
            soup = BeautifulSoup(reqtxt)
            images_soup = soup.find('div', {'class': 'product-images'})
            #print(images_soup)

                
            image_urls = []
            if images_soup:
                for img in images_soup.find_all('img'):
                    image_url = img.get('src')
                    if image_url:
                        if not image_url:
                            continue
                        if image_url.startswith('//'):
                            image_url = 'http:' + image_url
                        elif image_url.startswith('https'):
                            image_url = image_url
                        else:
                            main_url = f'https://vapewh.com{image_url}'
                        image_urls.append(image_url)
                    
            print(product_name, full_link)
            aside = soup.find('aside', {'class': 'product-aside'})

            
            # Extract the flavors
            flavor_select = soup.find('select', id='SingleOptionSelector-product-template-0')
            flavors = [option.text.strip() for option in flavor_select.find_all('option') if option.get('value') and option['value'] != 'pick-your'] if flavor_select else []

            # Extract stock availability information
            in_store_pickup = soup.find('span', class_='tomitInStock').text.strip() if soup.find('span', class_='tomitInStock') else None
            online_availability = soup.find('span', id='del-area').text.strip() if soup.find('span', id='del-area') else None


            description = ''
            warning = ''
            package_contents = ''
            ingredients = ''
            key_features = ''
            flavors = ''
            specifications = ''
            performance = ''
            disposable = ''
            why = ''
            preference = ''
            innovation = ''
            enjoyment = ''
            compatibility = ''
            coil = ''
            battery = ''
            puffs = ''
            nicotine = ''
            e_liquid = ''
            prefilled = ''
            devices = ''
            airflow = ''
            heating_element = ''
            operation = ''
            charging = ''
            screen = ''
            links_txt = ''
            colors = ''
            options_txt = ''
            returns = ''
            size = ''
            pods = ''
            hardware = ''
            wattage = ''
            faqs = ''

            section_strings_raw = {
                "FAQ's": "faqs",
                "FAQs": "faqs",
                "FAQ": "faqs",
                'Display Screen': 'screen',
                'Charging': 'charging',
                'Charging Port Info': 'charging',
                'Operation': 'operation',
                'Prefilled Capacity': 'prefilled',
                'Compatibility': 'compatibility',
                'Compatible with': 'compatibility',
                'Ingredients': 'ingredients',
                'INCLUDES': 'package_contents',
                'Kit Includes': 'package_contents',
                "What's Included": 'package_contents',
                'Package Content': 'package_contents',
                'Package Contents': 'package_contents',
                'Package Includes': 'package_contents',
                'Included in package': 'package_contents',
                'Included in the Package': 'package_contents',
                'Included': 'package_contents',
                'Includes': 'package_contents',
                'Contents': 'package_contents',
                "What’s Included": 'package_contents',
                'CALIFORNIA PROPOSITION 65 - Warning': 'warning',
                'CAUTION: this Product Contains Nicotine. Nicotine is an addictive chemical.': 'warning',
                'Warning': 'warning',
                'AVAILABLE OPTIONS': 'key_features',
                'Features and Specifications': 'key_features',
                'Key Features': 'key_features',
                'Features': 'key_features',
                'Flavors': 'flavors',
                'Flavor Selection': 'flavors',
                'Flavor Options': 'flavors',
                'Cutting-Edge Design and Technology:': 'key_features',
                'Superior Versatility and Performance:': 'key_features',
                'Flavor Selection:': 'flavors',
                'WARNING': 'warning',
                'Description': 'description',
                'Product Specifications': 'specifications',
                'Specifications': 'specifications',
                'Available Flavors': 'flavors',
                'Flavor Profiles': 'flavors',
                'Exceptional Flexibility and Performance': 'performance',
                'Disposable': 'disposable',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Tailored to Your Preference': 'preference',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Enjoy My Shisha on the Go': 'enjoyment',
                'The Ultimate Vaping Experience': 'enjoyment',
                'Elevate Your Vaping Experience': 'enjoyment',
                'Why Choose the Juice Head 30K Disposable?': 'why',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Features and Specifications': 'key_features',
                'Hotbox Features': 'key_features',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Experience the Ultimate Vaping Innovation': 'innovation',
                'Flavor Details': 'flavors',
                'Advanced Dual Mesh Coil': 'coil',
                'Coil Specifications': 'coil',
                'Battery Info': 'battery',
                'Puff Count': 'puffs',
                'Max Puffs': 'puffs',
                'What Is a Puff Count?': 'puffs',
                'Available Nicotine': 'nicotine',
                'Nicotine': 'nicotine',
                'Nicotine Strength': 'nicotine',
                'Battery Capacity': 'battery',
                'E-liquid Capacity': 'e_liquid',
                'Maximum Puffs': 'puffs',
                'Available Flavor Profiles': 'flavors',
                'Prefilled': 'prefilled',
                'Compatible Devices': 'devices',
                'Devices': 'devices',
                'California Proposition 65 Warning': 'warning',
                'Melli Flavors': 'flavors',
                'Airflow': 'airflow',
                'Heating Element': 'heating_element',
                'Quick Links': 'links',
                'Quick Link': 'links',
                'Colors': 'colors',
                'Color': 'colors',
                'Quick links': 'links',
                'Flavor': 'flavors',
                'Size': 'size',
                'Sizes': 'size',
                'Bottle Size': 'size',
                'Bottle Sizes': 'size',
                'Return and Exchange Policy Disclaimer:': 'returns',
                'Returns': 'returns',
                'Pods': 'pods',
                'Pod Info': 'pods',
                'Options': 'options',
                'Option': 'options',
                'Compatible Hardware': 'hardware',
                'Hardware': 'hardware',
                'Wattage Info': 'wattage',
                'Wattage': 'wattage',
                'Coil': 'coil',
                'Coils': 'coil',
                'Available Nicotine': 'nicotine',
                'How Are Disposables Different Than Others?': 'disposable',
            
            }
            
            functions = [
                str.upper,
                str.lower,
                str.capitalize,
                str.title,
                str.swapcase,
                str.casefold
            ]
            section_strings = dict()
            for k, v in section_strings_raw.items():
                key = k
                section_strings[key] = v
                if key.endswith(':'):
                    key = key.replace(':', '')
                    section_strings[key] = v
                else:
                    key = key.strip() + ':'
                    section_strings[key] = v
                for func in functions:
                    key = func(k)
                    section_strings[key] = v
                    if key.endswith(':'):
                        key = key.replace(':', '')
                        section_strings[key] = v
                    else:
                        key = key.strip() + ':'
                        section_strings[key] = v
                
            
            section_map = {
                'description': description,
                'warning': warning,
                'package_contents': package_contents,
                'ingredients': ingredients,
                'key_features': key_features,
                'flavors': flavors,
                'specifications': specifications,
                'performance': performance,
                'disposable': disposable,
                'why': why,
                'innovation': innovation,
                'preference': preference,
                'enjoyment': enjoyment,
                'compatibility': compatibility,
                'coil': coil,
                'battery': battery,
                'puffs': puffs,
                'nicotine': nicotine,
                'e_liquid': e_liquid,
                'prefilled': prefilled,
                'devices': devices,
                'airflow': airflow,
                'heating_element': heating_element,
                'operation': operation,
                'charging': charging,
                'screen': screen,
                'links': links_txt,
                'colors': colors,
                'returns': returns,
                'options': options_txt,
                'size': size,
                'pods': pods,
                'hardware': hardware,
                'wattage': wattage,
                'faqs': faqs
            }

            desc_section = soup.find('div', {'class': 'description'})
            if desc_section:
                cur_section = 'description'
                if desc_section:
                    headers = [tag.text for tag in desc_section.find_all(lambda tag: tag.name in ['strong','h1', 'h2', 'h3', 'h4', 'h5'])]
                    stext = desc_section.get_text(separator='\n').split('\n')
                    for s in stext:
                        s = s.strip()
                        if s == '':
                            continue
                        s_no_colon = s.replace(':', '').strip()
                        if s in section_strings:
                            cur_section = section_strings[s]
                        elif s_no_colon in section_strings:
                            cur_section = section_strings[s_no_colon]
                        elif s.lower().startswith('why') and s.endswith('?'):
                            cur_section = 'why'
                        elif 'features:' in s.lower():
                            cur_section = 'key_features'
                        elif s in headers:
                            all_headers.append(s)
                            header_counter = Counter(all_headers)
                            #if header_counter[s] > 5:
                            #    print(s, full_link)
                            cur_section = 'description'
                            section_map[cur_section] += f'{s}\n\n'
                        else:
                            section_map[cur_section] += f'{s}\n'
            # Find the details element containing "Flavor Descriptions"
            flavor_section = None

            # Loop through all details elements and find the one with the "Flavor Descriptions" header
            for details in soup.find_all('details'):
                summary_header = details.find('h3').text.strip() if details.find('h3') else None
                if summary_header == "Flavor Descriptions":
                    flavor_section = details.find('div', class_='metafield-rich_text_field')

            # Extract only the paragraphs related to flavor descriptions
            flavor_descriptions = []

            if flavor_section:
                flavor_paragraphs = flavor_section.find_all('p')
                for paragraph in flavor_paragraphs:
                    flavor_descriptions.append(paragraph.text.strip())
                    
            specifications_section = None

            # Loop through all details elements and find the right ones by header text
            for details in soup.find_all('details'):
                summary_header = details.find('h3').text.strip() if details.find('h3') else None
                if summary_header == "Specifications":
                    specifications_section = details.find('div', class_='metafield-rich_text_field')
                    
            if specifications_section:
                specifications = specifications_section.get_text().strip()
            else:
                specifications = None

            
            product_data = {
                'tag': tag,
                'sku': sku,
                "title": product_name,
#                 'brand': brand,
                "link": full_link,
                "price": price,
                'image_url': image_url,
                'image_urls': image_urls,
                'image_alt_text': alt_text,
                'rating': rating,
                'raters': raters,
                'variation': variation,
                'flavors': flavors,
                'flavors_section': flavor_descriptions,
                'in_store_pickup': in_store_pickup,
                'online_availability': online_availability,
                'specifications_section': specifications
            }
            
            if image_url:
                download_image(image_url, f'{tag}', save_dir='vapewh_images')
            
            image_num = 0
            for i in product_data['image_urls']:
                if i and i != '':
                    download_image(i, f'{tag}-image{image_num}', save_dir='vapewh_images')
                    image_num += 1
                    
            desc_fields = ''
            for s, v in section_map.items():
                if 'description' not in s:
                    s = s + '_description'
                product_data[s] = v.replace('\xa0', ' ').strip()
                if product_data[s] != '' and 'link' not in s:
                    desc_fields += f'\n{product_data[s]}'

            feats = list()
            #print(desc_fields)
            feat = find_features(desc_fields)
            any_found, puffs_res, nico_res, ml_res, flav_text, dev_text = feat

            if any_found:
                #print(feat)
                feats.append(feat)
            disposable,recharge,battery,mesh,usb,adjustable,found_flavs = features_to_cats(feats)
            product_data['disposable'] = disposable
            product_data['rechargeable'] = recharge
            product_data['battery'] = battery
            product_data['mesh'] = mesh
            product_data['usb'] = usb
            product_data['adjustable'] = adjustable
            
#             print(product_data)

            #print(product_data)


            
            product_list.append(product_data)
            product_tags.add(tag)
        
            if len(product_list) % 10 == 0:
                rc = (random.choice(product_list))
                print('{')
                for key, value in rc.items():
                    if not value:
                        continue

                    if isinstance(value, list):
                        print(f"\t{key}: {len(value)} items")
                    else:
                        value = value.replace('\n', ' ')

                        if key == 'image_url' or key == 'link' or key == 'title'  or key == 'tag' :
                            print(f"\t{key}: {value}")
                        elif 1 <= len(value) <= 20:
                            print(f"\t{key}: {value}")
                        elif len(value) > 20:
                            print(f"\t{key}: {value[:20]}...")
                print('}')
        
                print(len(product_list), 'ITEMS FOUND')
                print()
                print()
                for item, count in header_counter.most_common(50):
                    if count > 5:
                        print(f"{item}: {count}")
                print()
                print()

        save()
        #print(total_items, 'TOTAL ITEMS')

        if total_items == 0:
            break
    save()
save()

Pod Juice Salt: Banana Ice / Banana Freeze https://vapewh.com/products/pod-juice-salt-banana-ice
Tobacco Monster Salt Rich https://vapewh.com/products/tobacco-monster-salt-rich
PB & Jam Monster Strawberry https://vapewh.com/products/pb-jam-monster-strawberry
Frozen Fruit Monster Strawberry Banana Ice https://vapewh.com/products/frozen-fruit-monster-strawberry-banana-ice
SAVING...
SAVING...
LINK INDEX 40 OF 45  -- PAGE 1 OF 2 (15) https://vapewh.com/collections/pod-devices?page=1
FreeMax GEMM Disposable Tanks https://vapewh.com/products/freemax-disposable-tanks
UWELL CALIBURN AK3 Pod Kit | Free Shipping https://vapewh.com/products/uwell-caliburn-ak3-pod-kit-gray
VOOPOO PnP Pod Tank https://vapewh.com/products/voopoo-pnp-pod-tank
{
	tag: kado-bar-5000-puff-disposable
	sku: MASTER-KB5-CA
	title: Kado Bar 5000 Puff Disposable Vape
	link: https://vapewh.com/products/kado-bar-5000-puff-disposable
	image_url: http://vapewh.com/cdn/shop/files/Kado_Bar_10k_BE_Virginia_Tobacco.jpg?v=1720751861&w

SAVING...
LINK INDEX 42 OF 45  -- PAGE 1 OF 2 (12) https://vapewh.com/collections/tanks-coils?page=1
Smok Baby Beast Coil (V8-Q2 0.4ohm) https://vapewh.com/products/baby-beast-coil-v8-q2-0-4-ohm
510 Drip Tips - Full Resin https://vapewh.com/products/510-drip-tips-full-resin
SAVING...
LINK INDEX 42 OF 45  -- PAGE 2 OF 2 (2) https://vapewh.com/collections/tanks-coils?page=2
SAVING...
SAVING...
LINK INDEX 43 OF 45  -- PAGE 1 OF 2 (8) https://vapewh.com/collections/replacement-pods?page=1
SAVING...
LINK INDEX 43 OF 45  -- PAGE 2 OF 2 (2) https://vapewh.com/collections/replacement-pods?page=2
SAVING...
SAVING...
LINK INDEX 44 OF 45  -- PAGE 1 OF 2 (29) https://vapewh.com/collections/ri-products?page=1
SAVING...
LINK INDEX 44 OF 45  -- PAGE 2 OF 2 (30) https://vapewh.com/collections/ri-products?page=2
SAVING...
SAVING...
SAVING...
